In [305]:
#各年のレース情報取得

import requests
from bs4 import BeautifulSoup
import csv
import re
import time
import pandas
import os

#ディレクトリ作成 (好きなディレクトリを指定)
os.makedirs("./競馬/arima2021/raceInfo/")

#過去20年のレースIDの取得
raceid_list = []
#URLを変更することで有馬記念以外のレースも取得できるはず
url = "https://db.netkeiba.com/?pid=race_list&word=%5E%CD%AD%C7%CF%B5%AD%C7%B0"
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")

soup_txt_race = soup.find_all(href = re.compile("/race/20"))
for num in range(20):
    raceid_list.append(soup_txt_race[num].attrs['href'])

#過去20年のレースのデータを取得
for count,i in enumerate(raceid_list):
    url = "https://db.netkeiba.com" + i
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    soup_span = soup.find_all("span")
    len(soup_span)

    #頭数
    allnum = int((len(soup_span) - 8) / 3)

    #馬の情報を以下で取得
    soup_txt_l = soup.find_all(class_ = "txt_l")

    #馬の名前
    name = []
    for num in range(allnum):
        name.append(soup_txt_l[4 * num].contents[1].contents[0])

    #騎手名
    jockey = []
    for num in range(allnum):
        jockey.append(soup_txt_l[4 * num + 1].contents[1].contents[0])

    #馬番
    soup_txt_r = soup.find_all(class_ = "txt_r")
    horse_number = []
    for num in range(allnum):
        horse_number.append(soup_txt_r[1 + 5 * num].contents[0])

    #走破タイム
    runtime = [] 
    for num in range(allnum):
        try:
            runtime.append(soup_txt_r[2 + num * 5].contents[0])
        except IndexError:
            runtime.append(None)

    #オッズ
    odds = []
    for num in range(allnum):
        try:
            odds.append(soup_txt_r[3 + 5 * num].contents[0])
        except IndexError:
            odds.append(None)

    #通過順
    soup_nowrap = soup.find_all("td",nowrap = "nowrap",class_ = None)
    pas = []
    for num in range(allnum):
        try:
            pas.append(soup_nowrap[3 * num].contents[0])
        except IndexError:
            pas.append(None)

    #体重
    weight = []
    for num in range(allnum):
        try:
            weight.append(soup_nowrap[3 * num + 1].contents[0])
        except IndexError:
            weight.append(None)

    #性齢
    soup_tet_c = soup.find_all("td",nowrap = "nowrap",class_  = "txt_c")
    sex_old = []
    for num in range(allnum):
        sex_old.append(soup_tet_c[6 * num].contents[0])

    #斤量
    handi = []
    for num in range(allnum):
        handi.append(soup_tet_c[6 * num + 1].contents[0])

    #上がり
    last = []
    for num in range(allnum):
        try:
            last.append(soup_tet_c[6 * num + 3].contents[0].contents[0])
        except IndexError:
            last.append(None)

    #人気
    pop = []
    for num in range(allnum):
        try:
            pop.append(soup_span[3 * num + 10].contents[0])
        except IndexError:
            pop.append(None)
    
    #データ格納
    houseInfo = [name,jockey,horse_number,runtime,odds,pas,weight,sex_old,handi,last,pop]
    
    #CSV書き出し
    #ファイルパス指定
    year = 2021-count    
    filepass1 = "./競馬/arima2021/raceInfo/arima{}_test.csv"
    filepass2 = filepass1.format(year)
    with open(filepass2, 'a', newline = '',encoding = "SHIFT-JIS") as f:
        csv.writer(f).writerows(houseInfo)
    col_num = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
    df = pandas.read_csv(filepass2,encoding = "SHIFT-JIS",names = col_num)
    
    #列名追加
    df_mod = df.rename(index = {0:"馬名",1:"騎手名",2:"枠順",3:"走破タイム",4:"オッズ",5:"通過順位",6:"馬体重",7:"性齢",8:"斤量",9:"上がり3ハロン",10:"人気"})
  
    #ファイル書き出し
    filepass3 = "./競馬/arima2021/raceInfo/arima{}.csv"
    filepass4 = filepass3.format(year)
    df_mod.to_csv(filepass4)
    
    #testファイルの削除
    os.remove(filepass2)
    
    #間隔
    time.sleep(1)

In [356]:
#各年の出走馬のデータ取得

import requests
from bs4 import BeautifulSoup
import csv
import re
import time
import pandas
import os

#関数定義
#列名から部分一致でindexを返す関数の定義
def inclusive_index(lst, purpose):
    for i, e in enumerate(lst):
        if purpose in e: return i

    raise IndexError

#ディレクトリ作成
#os.makedirs("./競馬/arima2021/horseInfo/")

#過去20年のレースIDの取得
baseurl = "https://db.netkeiba.com/?pid=race_list&word=%5E%CD%AD%C7%CF%B5%AD%C7%B0"
r1= requests.get(baseurl)
soup1 = BeautifulSoup(r1.content, "html.parser")
soup1_txt_race = soup1.find_all(href = re.compile("/race/20"))

#レースidと検索用パラメータの格納
race_para_list=[]
raceid_list = []
for num in range(20):
    race_para_list.append(soup1_txt_race[num])
    raceid_list.append(soup1_txt_race[num].attrs['href'])

#過去20年のレースのデータを取得
for count,i in enumerate(raceid_list):
    raceurl = "https://db.netkeiba.com" + i
    r2 = requests.get(raceurl)
    soup2 = BeautifulSoup(r2.content, "html.parser")
    soup2_span = soup2.find_all("span")
    len(soup2_span)
    #出走頭数
    allnum = int((len(soup2_span) - 8) / 3)

    #出走馬のページURL取得
    soup2_txt_h = soup2.find_all(href = re.compile("/horse/"))
    #出走馬のURL
    horse_url_list = [] 
    for num in range(allnum):
        horse_url_list.append(soup2_txt_h[num].attrs['href'])
        
    #出走馬のデータ取得
    for horse_url in horse_url_list:
        horseurl = "https://db.netkeiba.com"+horse_url
        r3 = requests.get(horseurl)
        soup3 = BeautifulSoup(r3.content, "html.parser")

        # 直近3回の出走レース名の取得
        soup3_txt_race = soup3.find_all(href = re.compile("/race/20"))
        soup3_txt_race_str = [str(n) for n in soup3_txt_race]
        idx=soup3_txt_race_str.index(str(race_para_list[count]))
        recent_race=[soup3_txt_race[idx+1],soup3_txt_race[idx+2],soup3_txt_race[idx+3]]
        recent_race_str=[str(n) for n in recent_race] 

        recent_race_list = []
        for num2 in range(0,3):
            try:
                recent_race_list.append(recent_race[num2].contents[0])
            except IndexError:
                recent_race_list.append(None)

        #直近3回の出走レースの詳細情報の取得 
        soup3_td = soup3.find_all('td')
        soup3_td_str = [str(n) for n in soup3_td]

        recent_race_info = []
        for race in  recent_race_str:
            try:
                idx2=inclusive_index(soup3_td_str,race)
                recent_race_info.append(race)
                recent_race_info.append(soup3_td[idx2+5].text)
                #recent_race_info.append(soup3_td[idx2+6].text)
                recent_race_info.append(soup3_td[idx2+7].text)
            except IndexError:
                recent_race_info.append(None)
       #print(recent_race_info)
            
        #みんなの評価の取得
        soup_txt_review = soup3.find_all(src = re.compile("https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/review_bar_"))
        #芝適正(値が大きいほどダート適正)
        turf_type = soup_txt_review[1].get("width")
        #距離適性(値が大きいほど長距離適性)
        dist_type = soup_txt_review[3].get("width")
        #脚質(値が大きいほど追い込み)
        run_type = soup_txt_review[5].get("width")
        #成長(値が大きいほど晩成)
        grow_type = soup_txt_review[7].get("width")
        #馬場適性(値が大きいほど重馬場苦手)
        field_type = soup_txt_review[9].get("width")

        #馬の総合評価
        soup_stars = soup3.find_all(class_ = re.compile("star"))
        #総合評価
        try:
            soup_all_stars = soup_stars[0].contents[0].contents[0]
        except IndexError:
            soup_all_stars = ""

        #実績評価
        try:
            soup_result_stars = soup_stars[1].contents[0]
        except IndexError:
            soup_result_stars = ""
        #ポテンシャル評価
        try:
            soup_potential_stars = soup_stars[2].contents[0].contents[0]
        except IndexError:
            soup_potential_stars = ""
            
            
        #全ての結果を結合
        eachhorseInfo = [turf_type,dist_type,run_type,grow_type,field_type,soup_all_stars,soup_result_stars,soup_potential_stars]+recent_race_info
        #不正な文字コードを削除
        eachhorseInfo_mod = []
        for item in eachhorseInfo:
            item_mod = item.replace("\xa0","") 
            eachhorseInfo_mod.append(item_mod)
            
        #CSVに書き出し
        year = 2021-count
        filepass1 = "./競馬/arima2021/horseInfo/arimahorse{}_test.csv"
        filepass2 = filepass1.format(year)
        with open(filepass2, 'a',newline = '',encoding = "SHIFT-JIS") as f:
            csv.writer(f).writerow(eachhorseInfo_mod)
    
#csvの整理
    col_names=["芝適性","距離適性","脚質","成長","馬場適性","総合評価","実績評価","ポテンシャル評価","前走レース名","前走オッズ","前走成績","2走前レース名","2走前オッズ","2走前成績","3走前レース名","3走前オッズ","3走前成績"]
    df = pandas.read_csv(filepass2,encoding = "SHIFT-JIS",names=col_names)
    year = 2021-count
    filepass3 = "./競馬/arima2021/horseInfo/arimahorse{}.csv"
    filepass4 = filepass3.format(year)
    #転置
    df.T.to_csv(filepass4)
    #testファイルの削除
    os.remove(filepass2)
    
#確認
pandas.read_csv("/Users/yamagamihiroki/競馬　ラップ分析/keiba/競馬/arima2021/horseInfo/arimahorse2020.csv",index_col=0,encoding = "utf-8")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
芝適性,1,8,3,2,1,1,11,6,1,3,8,5,1,1,1,2
距離適性,63,58,99,68,85,83,48,93,116,70,99,91,58,101,97,82
脚質,76,104,111,44,106,26,75,79,1,99,116,42,1,62,74,71
成長,91,87,74,70,86,37,40,116,97,95,108,82,116,105,111,78
馬場適性,9,53,75,37,53,18,82,19,58,88,70,11,1,22,20,25
総合評価,4.35,4.08,4.36,4.3,4.22,4.23,4.04,4.03,4.13,4.39,3.67,4.4,3.5,4.21,3.69,4.17
実績評価,4.35,4.08,4.36,4.3,4.22,4.23,4.04,4.03,4.13,4.39,3.67,4.4,3.5,4.21,3.69,4.17
ポテンシャル評価,4.31,3.68,4.33,4.21,3.66,3.68,3.95,3.37,4.0,4.0,3.43,4.11,3.17,3.73,3.85,4.04
前走レース名,"<a href=""/race/202005040811/"" title=""天皇賞(秋)(G1...","<a href=""/race/202009050411/"" title=""エリザベス女王杯(...","<a href=""/race/202005040811/"" title=""天皇賞(秋)(G1...","<a href=""/race/202009050411/"" title=""エリザベス女王杯(...","<a href=""/race/202005050912/"" title=""ジャパンC(G1)...","<a href=""/race/202005050912/"" title=""ジャパンC(G1)...","<a href=""/race/202009050611/"" title=""マイルチャンピオン...","<a href=""/race/202005050912/"" title=""ジャパンC(G1)...","<a href=""/race/202006050111/"" title=""ステイヤーズS(G...","<a href=""/race/202009050411/"" title=""エリザベス女王杯(...","<a href=""/race/202005050912/"" title=""ジャパンC(G1)...","<a href=""/race/202005050912/"" title=""ジャパンC(G1)...","<a href=""/race/202008040611/"" title=""菊花賞(G1)"">...","<a href=""/race/202005050211/"" title=""アルゼンチン共和国...","<a href=""/race/202009030811/"" title=""宝塚記念(G1)""...","<a href=""/race/202005040811/"" title=""天皇賞(秋)(G1..."
前走オッズ,4.4,12.3,17.4,3.3,45.4,24.9,165.8,432.9,13.8,5.5,101.1,44.6,12.5,5.3,106.1,32.3


In [363]:
#レースデータと出走馬のデータの結合
import requests
from bs4 import BeautifulSoup
import csv
import re
import time
import pandas
import os

#ディレクトリ作成
os.makedirs("./競馬/arima2021/allInfo/")

for year in range(2002,2022):
    racepass = "./競馬/arima2021/raceInfo/arima" + str(year) + ".csv"
    horsepass = "./競馬/arima2021/horseInfo/arimahorse" + str(year) + ".csv"
    df1 = pandas.read_csv(racepass)
    df2 = pandas.read_csv(horsepass)
    df_concat = pandas.concat([df1,df2], axis = 0, ignore_index = False)
    allInfopass_tmp = "./競馬/arima2021/allInfo/arima{}.csv"
    allInfopass = allInfopass_tmp.format(year)
    df_concat.to_csv(allInfopass, index = False)

In [364]:
#確認
#pandas.read_csv("/Users/yamagamihiroki/競馬　ラップ分析/keiba/競馬/arima2021/horseInfo/arimahorse2009.csv",index_col=0,encoding = "utf-8")
pandas.read_csv("/Users/yamagamihiroki/競馬　ラップ分析/keiba/競馬/arima2021/allInfo/arima2021.csv",index_col=0,encoding = "utf-8")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
馬名,エフフォーリア,ディープボンド,クロノジェネシス,ステラヴェローチェ,タイトルホルダー,アリストテレス,アカイイト,モズベッロ,ユーキャンスマイル,キセキ,ウインキートス,シャドウディーヴァ,パンサラッサ,ペルシアンナイト,メロディーレーン,アサマノイタズラ
騎手名,横山武史,和田竜二,ルメール,Ｍ．デム,横山和生,武豊,幸英明,池添謙一,藤岡佑介,松山弘平,丹内祐次,横山典弘,菱田裕二,Ｃ．デム,岩田望来,田辺裕信
枠順,10,5,7,9,16,11,13,3,8,15,6,12,2,1,4,14
走破タイム,2:32.0,2:32.1,2:32.2,2:32.3,2:32.5,2:33.0,2:33.1,2:33.5,2:33.6,2:33.6,2:34.3,2:34.3,2:34.4,2:34.6,2:35.2,2:35.4
オッズ,2.1,20.9,2.9,7.9,10.2,47.7,31.8,146.2,150.6,43.8,70.0,144.5,45.5,62.3,146.8,51.1
通過順位,9-9-8-5,5-6-6-5,7-7-8-8,10-11-10-8,2-2-2-2,15-15-14-12,12-12-13-10,13-13-10-13,14-14-14-15,7-7-6-3,3-3-3-5,3-4-5-10,1-1-1-1,5-4-3-3,10-10-10-13,16-16-16-16
馬体重,516(+2),502(0),478(0),498(+4),476(+12),474(-6),516(+2),488(0),512(+6),516(+4),480(+10),476(-2),480(+6),500(-2),352(-2),496(+4)
性齢,牡3,牡4,牝5,牡3,牡3,牡4,牝4,牡5,牡6,牡7,牝4,牝5,牡4,牡7,牝5,牡3
斤量,55,57,55,55,55,57,55,57,57,57,55,55,57,57,55,55
上がり3ハロン,35.9,36.1,36.0,35.9,36.9,36.4,36.6,37.1,36.9,37.6,38.5,38.4,39.1,38.8,38.8,38.4


In [ ]:
#各年の出走馬のデータ取得

import requests
from bs4 import BeautifulSoup
import csv
import re
import time
import pandas
import os

#ディレクトリ作成
os.makedirs("./競馬/arima2021/horseInfo/")

#過去20年のレースIDの取得
raceid_list = []
url = "https://db.netkeiba.com/?pid=race_list&word=%5E%CD%AD%C7%CF%B5%AD%C7%B0"
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")

soup_txt_race = soup.find_all(href = re.compile("/race/20"))
for num in range(20):
    raceid_list.append(soup_txt_race[num].attrs['href'])

#過去20年のレースのデータを取得
for count,i in enumerate(raceid_list):
    url = "https://db.netkeiba.com" + i
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    soup_span = soup.find_all("span")
    len(soup_span)

    #頭数
    allnum = int((len(soup_span) - 8) / 3)

    #出走馬のページURL取得
    soup_txt_h = soup.find_all(href = re.compile("/horse/"))
    #出走馬のURL
    horse_url_list = [] 
    for num in range(allnum):
        horse_url_list.append(soup_txt_h[num].attrs['href'])

    for horse_url in horse_url_list:
        #出走馬のデータ取得
        url = "https://db.netkeiba.com"+horse_url
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        soup_span = soup.find_all("span")

        #過去2年の出走レース取得
        year = 2021 - count 
        racepass1 = "/race/{}|/race/{}"
        racepass2 = racepass1.format(year-1,year-2)
        soup_txt_race = soup.find_all(href = re.compile(racepass2))
        #print( soup_txt_race)
        #過去3回の出走レース名取得
        race_list = []
        for num in range(1,3):
            try:
                race_list.append(soup_txt_race[num].contents[0])
            except IndexError:
                race_list.append(None)

        #過去の成績取得
        soup_txt_result = soup.find_all(class_ = "txt_right")

        #過去3戦の成績
        rank_result = []
        for num in range(0,3):
            try:
                rank_result.append(soup_txt_result[6 + 11*num].contents[0])
            except IndexError:
                rank_result.append(None)
        #rank_result


        #過去3戦の人気取得
        pop_result = []
        for num in range(0,3):
            try:
                pop_result.append(soup_txt_result[5 + 11*num].contents[0])
            except IndexError:
                pop_result.append(None)
        #pop_result

        #過去レースのオッズ
        odds_result = []
        for num in range(0,3):
            try:
                odds_result.append(soup_txt_result[4 + 11*num].contents[0])
            except IndexError:
                odds_result.append(None)
        #odds_result

        
        

        #みんなの評価の取得
        soup_txt_review = soup.find_all(src = re.compile("https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/review_bar_"))
        #芝適正(値が大きいほどダート適正)
        turf_type = soup_txt_review[1].get("width")
        #距離適性(値が大きいほど長距離適性)
        dist_type = soup_txt_review[3].get("width")
        #脚質(値が大きいほど追い込み)
        run_type = soup_txt_review[5].get("width")
        #成長(値が大きいほど晩成)
        grow_type = soup_txt_review[7].get("width")
        #馬場適性(値が大きいほど重馬場苦手)
        field_type = soup_txt_review[9].get("width")

        #馬の総合評価
        soup_stars = soup.find_all(class_ = re.compile("star"))
        #総合評価
        try:
            soup_all_stars = soup_stars[0].contents[0].contents[0]
        except IndexError:
            soup_all_stars = ""

        #実績評価
        try:
            soup_result_stars = soup_stars[1].contents[0]
        except IndexError:
            soup_result_stars = ""
        #ポテンシャル評価
        try:
            soup_potential_stars = soup_stars[2].contents[0].contents[0]
        except IndexError:
            soup_potential_stars = ""


        eachhorseInfo = [turf_type,dist_type,run_type,grow_type,field_type,soup_all_stars,soup_result_stars,soup_potential_stars]
        #過去3戦の結果格納
        for num in rank_result:
            eachhorseInfo.append(num)

        #過去3戦のオッズ格納
        for num in odds_result:
            eachhorseInfo.append(num)
        #不正な文字コードを削除
        eachhorseInfo_mod = []
        for item in eachhorseInfo:
            item_mod = item.replace("\xa0","") 
            eachhorseInfo_mod.append(item_mod)
        #print(eachhorseInfo_mod)
        
        #CSVに書き出し
        year = 2021-count
        filepass1 = "./競馬/arima2021/horseInfo/arimahorse{}_test.csv"
        filepass2 = filepass1.format(year)
        with open(filepass2, 'a',newline = '',encoding = "SHIFT-JIS") as f:
            csv.writer(f).writerow(eachhorseInfo_mod)
    col_names = ["芝適性","距離適性","脚質","成長","馬場適性","総合評価","実績評価",
               "ポテンシャル評価","前走成績","前々走成績","前々前走成績","前走オッズ","前々走オッズ","前々前走オッズ"]
    df = pandas.read_csv(filepass2,encoding = "SHIFT-JIS",names = col_names)
    year = 2021-count
    filepass3 = "./競馬/arima2021/horseInfo/arimahorse{}.csv"
    filepass4 = filepass3.format(year)
    
    #転置
    df.T.to_csv(filepass4)
    #testファイルの削除
    os.remove(filepass2)


In [3]:
#レースデータと出走馬のデータの結合
import requests
from bs4 import BeautifulSoup
import csv
import re
import time
import pandas
import os

#ディレクトリ作成
os.makedirs("./競馬/arima2021/allInfo/")

for year in range(2002,2021):
    racepass = "./競馬/arima2021/raceInfo/arima" + str(year) + ".csv"
    horsepass = "./競馬/arima2021/horseInfo/arimahorse" + str(year) + ".csv"
    df1 = pandas.read_csv(racepass)
    df2 = pandas.read_csv(horsepass)
    df_concat = pandas.concat([df1,df2], axis = 0, ignore_index = False)
    allInfopass_tmp = "./競馬/arima2021/allInfo/arima{}.csv"
    allInfopass = allInfopass_tmp.format(year)
    df_concat.to_csv(allInfopass, index = False)

In [279]:
import pandas
#CSV確認(年によって変える)
#pandas.read_csv("/Users/01036204/Desktop/競馬/arima2021/allInfo/arima2001.csv",index_col=0)

pandas.read_csv("/Users/yamagamihiroki/競馬　ラップ分析/keiba/競馬/arima2022/horseInfo/arimahorse2021.csv",index_col=0)
#pandas.read_csv("/Users/yamagamihiroki/競馬　ラップ分析/keiba/競馬/arima2021/raceInfo/arima2004.csv",index_col=0)
#pandas.read_csv("/Users/yamagamihiroki/競馬　ラップ分析/keiba/競馬/arima2021/allInfo/arima2011.csv",index_col=0)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
1,1,1,3,1,1,6,1,8,5,1,1,1,11,1,1
72,113,63,65,109,111,97,97,99,91,82,102,69,48,114,58
24,13,76,113,1,43,99,74,116,42,39,97,1,75,109,91
54,96,91,77,96,84,91,111,108,82,82,58,113,40,102,58
34,8,9,6,7,64,10,20,70,11,58,39,7,82,35,44
4.48,4.18,4.35,4.41,4.73,4.13,3.54,3.69,3.67,4.4,3.64,3.64,4.09,4.04,3.61,3.06
4.48.1,4.18,4.35,4.41,4.73,4.13,3.54,3.69,3.67,4.4,3.64,3.64,4.09,4.04,3.61,3.06
4.53,3.95,4.31,4.07,4.61,3.87,3.33,3.85,3.43,4.11,3.5,2.57,3.73,3.95,2.88,2.56
"<a href=""/race/202105040811/"" title=""天皇賞(秋)(G1)"">天皇賞(秋)(G1)</a>","<a href=""/race/2021C8100304/"" title=""凱旋門賞(G1)""...","<a href=""/race/2021C8100304/"" title=""凱旋門賞(G1)""...","<a href=""/race/202109040611/"" title=""菊花賞(G1)"">...","<a href=""/race/202109040611/"" title=""菊花賞(G1)"">...","<a href=""/race/202105050812/"" title=""ジャパンC(G1)...","<a href=""/race/202109050411/"" title=""エリザベス女王杯(...","<a href=""/race/202105050812/"" title=""ジャパンC(G1)...","<a href=""/race/202105050812/"" title=""ジャパンC(G1)...","<a href=""/race/202105050812/"" title=""ジャパンC(G1)...","<a href=""/race/202109050411/"" title=""エリザベス女王杯(...","<a href=""/race/202105050812/"" title=""ジャパンC(G1)...","<a href=""/race/202103020411/"" title=""福島記念(G3)""...","<a href=""/race/202109060111/"" title=""チャレンジC(G3...","<a href=""/race/202109040809/"" title=""古都S(3勝クラス...","<a href=""/race/202109040611/"" title=""菊花賞(G1)"">..."
3.4,12.8,4.8,4.1,8.0,20.5,64.9,278.8,240.1,37.4,9.7,76.4,9.0,10.0,5.3,13.1


In [277]:
import pandas
import os
os.chdir("/Users/01036204/Desktop/")
allInfopass1 = "./競馬/arima2021/allInfo/arima2001.csv"
allInfopass2 = "./競馬/arima2021/allInfo/arima2002.csv"
df1 = pandas.read_csv(allInfopass1 , header = 1)
df2 = pandas.read_csv(allInfopass2 , header = 1)
df_concat = pandas.concat([df1,df2], axis = 1, ignore_index = False)
df_concat.to_csv( "./競馬/arima2021/arima20012002.csv", index = False)
pandas.read_csv("/Users/01036204/Desktop/競馬/arima2021/arima20012002.csv",index_col=0,header=None)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/01036204/Desktop/'

,0,1,2,3,4,5,6,7,8,9,...,6.1,7.1,8.1,9.1,10.1,11.1,12.1,13.1,14.1,15.1
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
馬名,マンハッタンカフェ,アメリカンボス,トゥザヴィクトリー,メイショウドトウ,テイエムオペラオー,テイエムオーシャン,シンコウカリド,メイショウオウドウ,トウカイオーザ,ナリタトップロード,...,ジャングルポケット,フサイチランハート,エアシャカール,テイエムオーシャン,ヒシミラクル,アクティブバイオ,アメリカンボス,イーグルカフェ,NaN,NaN
騎手名,蛯名正義,江田照男,武豊,安田康彦,和田竜二,本田優,田中勝春,飯田祐史,Ｍデムー,渡辺薫彦,...,藤田伸二,バルジュ,横山典弘,本田優,角田晃一,後藤浩輝,江田照男,田中勝春,NaN,NaN
枠順,4,1,2,13,12,8,9,7,10,5,...,9,13,4,5,3,14,7,10,NaN,NaN
走破タイム,2:33.1,2:33.3,2:33.3,2:33.3,2:33.3,2:33.5,2:33.7,2:33.7,2:33.7,2:33.9,...,2:33.9,2:33.9,2:34.1,2:34.1,2:34.2,2:34.4,2:34.7,2:34.9,NaN,NaN
オッズ,7.1,116.9,17.7,5.5,1.8,26.8,24.2,84.4,15.4,7.5,...,4.5,94.9,21.1,33.2,18.5,72.6,62.7,29.3,NaN,NaN
通過順位,7-8-11-9,3-3-2-2,1-1-1-1,5-6-3-4,7-8-7-6,9-6-13-11,2-2-3-2,5-4-9-6,12-12-9-11,9-10-11-11,...,8-8-6-4,11-12-12-11,13-13-10-11,6-6-6-6,13-14-14-11,4-4-4-6,8-8-6-9,11-11-12-14,NaN,NaN
馬体重,504(+10),486(+6),488(-2),512(+4),480(0),440(-6),520(+20),474(0),492(+14),494(+4),...,476(-2),494(+6),510(+2),468(+6),464(+4),502(+2),492(+6),486(+14),NaN,NaN
性齢,牡3,牡6,牝5,牡5,牡5,牝3,牡3,牡6,牡4,牡5,...,牡4,牡5,牡5,牝4,牡3,牡5,牡7,牡5,NaN,NaN
斤量,55,57,55,57,57,53,55,57,57,57,...,57,57,57,55,55,57,57,57,NaN,NaN


[<a href="/race/201806050811/" title="有馬記念(G1)">有馬記念(G1)</a>, <a href="/race/201808040711/" title="菊花賞(G1)">菊花賞(G1)</a>, <a href="/race/201804021211/" title="新潟記念(G3)">新潟記念(G3)</a>, <a href="/race/201805021210/" title="東京優駿(G1)">東京優駿(G1)</a>, <a href="/race/201809020111/" title="毎日杯(G3)">毎日杯(G3)</a>, <a href="/race/201805010409/" title="ゆりかもめ賞(500万下)">ゆりかもめ賞(500万下)</a>, <a href="/race/201705050605/" title="2歳新馬">2歳新馬</a>]


In [ ]:
#各年の出走馬のデータ取得

import requests
from bs4 import BeautifulSoup
import csv
import re
import time
import pandas
import os

#ディレクトリ作成
os.makedirs("./競馬/arima2021/horseInfo/")

#過去20年のレースIDの取得
raceid_list = []
url = "https://db.netkeiba.com/?pid=race_list&word=%5E%CD%AD%C7%CF%B5%AD%C7%B0"
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")

soup_txt_race = soup.find_all(href = re.compile("/race/20"))
for num in range(20):
    raceid_list.append(soup_txt_race[num].attrs['href'])

#過去20年のレースのデータを取得
for count,i in enumerate(raceid_list):
    url = "https://db.netkeiba.com" + i
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    soup_span = soup.find_all("span")
    len(soup_span)

    #頭数
    allnum = int((len(soup_span) - 8) / 3)

    #出走馬のページURL取得
    soup_txt_h = soup.find_all(href = re.compile("/horse/"))
    #出走馬のURL
    horse_url_list = [] 
    for num in range(allnum):
        horse_url_list.append(soup_txt_h[num].attrs['href'])

    for horse_url in horse_url_list:
        #出走馬のデータ取得
        url = "https://db.netkeiba.com"+horse_url
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        soup_span = soup.find_all("span")

        #過去2年の出走レース取得
        year = 2021 - count 
        racepass1 = "/race/{}|/race/{}"
        racepass2 = racepass1.format(year-1,year-2)
        soup_txt_race = soup.find_all(href = re.compile(racepass2))
        #print( soup_txt_race)
        #過去3回の出走レース名取得
        race_list = []
        for num in range(1,3):
            try:
                race_list.append(soup_txt_race[num].contents[0])
            except IndexError:
                race_list.append(None)

        #過去の成績取得
        soup_txt_result = soup.find_all(class_ = "txt_right")

        #過去3戦の成績
        rank_result = []
        for num in range(0,3):
            try:
                rank_result.append(soup_txt_result[6 + 11*num].contents[0])
            except IndexError:
                rank_result.append(None)
        #rank_result


        #過去3戦の人気取得
        pop_result = []
        for num in range(0,3):
            try:
                pop_result.append(soup_txt_result[5 + 11*num].contents[0])
            except IndexError:
                pop_result.append(None)
        #pop_result

        #過去レースのオッズ
        odds_result = []
        for num in range(0,3):
            try:
                odds_result.append(soup_txt_result[4 + 11*num].contents[0])
            except IndexError:
                odds_result.append(None)
        #odds_result


        #みんなの評価の取得
        soup_txt_review = soup.find_all(src = re.compile("https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/review_bar_"))
        #芝適正(値が大きいほどダート適正)
        turf_type = soup_txt_review[1].get("width")
        #距離適性(値が大きいほど長距離適性)
        dist_type = soup_txt_review[3].get("width")
        #脚質(値が大きいほど追い込み)
        run_type = soup_txt_review[5].get("width")
        #成長(値が大きいほど晩成)
        grow_type = soup_txt_review[7].get("width")
        #馬場適性(値が大きいほど重馬場苦手)
        field_type = soup_txt_review[9].get("width")

        #馬の総合評価
        soup_stars = soup.find_all(class_ = re.compile("star"))
        #総合評価
        try:
            soup_all_stars = soup_stars[0].contents[0].contents[0]
        except IndexError:
            soup_all_stars = ""

        #実績評価
        try:
            soup_result_stars = soup_stars[1].contents[0]
        except IndexError:
            soup_result_stars = ""
        #ポテンシャル評価
        try:
            soup_potential_stars = soup_stars[2].contents[0].contents[0]
        except IndexError:
            soup_potential_stars = ""


        eachhorseInfo = [turf_type,dist_type,run_type,grow_type,field_type,soup_all_stars,soup_result_stars,soup_potential_stars]
        #過去3戦の結果格納
        for num in rank_result:
            eachhorseInfo.append(num)

        #過去3戦のオッズ格納
        for num in odds_result:
            eachhorseInfo.append(num)
        #不正な文字コードを削除
        eachhorseInfo_mod = []
        for item in eachhorseInfo:
            item_mod = item.replace("\xa0","") 
            eachhorseInfo_mod.append(item_mod)
        #print(eachhorseInfo_mod)
        
        #CSVに書き出し
        year = 2021-count
        filepass1 = "./競馬/arima2021/horseInfo/arimahorse{}_test.csv"
        filepass2 = filepass1.format(year)
        with open(filepass2, 'a',newline = '',encoding = "SHIFT-JIS") as f:
            csv.writer(f).writerow(eachhorseInfo_mod)
    col_names = ["芝適性","距離適性","脚質","成長","馬場適性","総合評価","実績評価",
               "ポテンシャル評価","前走成績","前々走成績","前々前走成績","前走オッズ","前々走オッズ","前々前走オッズ"]
    df = pandas.read_csv(filepass2,encoding = "SHIFT-JIS",names = col_names)
    year = 2021-count
    filepass3 = "./競馬/arima2021/horseInfo/arimahorse{}.csv"
    filepass4 = filepass3.format(year)
    
    #転置
    df.T.to_csv(filepass4)
    #testファイルの削除
    os.remove(filepass2)


In [43]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import time
import pandas
import os

#ディレクトリ作成
#os.makedirs("./競馬/arima2021/horseInfo/")

#過去20年のレースIDの取得
raceid_list = []
url = "https://db.netkeiba.com/?pid=race_list&word=%5E%CD%AD%C7%CF%B5%AD%C7%B0"
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")
soup_txt_race = soup.find_all(href = re.compile("/race/20"))
for num in range(1):
    raceid_list.append(soup_txt_race[num].attrs['href'])

#過去20年のレースのデータを取得
for count,i in enumerate(raceid_list):
    url = "https://db.netkeiba.com" + i
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    soup_span = soup.find_all("span")
    len(soup_span)
    #頭数
    allnum = int((len(soup_span) - 8) / 3)
    #出走馬のページURL取得
    soup_txt_h = soup.find_all(href = re.compile("/horse/"))
    #出走馬のURL
    horse_url_list = [] 
    for num in range(allnum):
        horse_url_list.append(soup_txt_h[num].attrs['href'])
    print(horse_url_list )

['/horse/2018105027/', '/horse/2017102170/', '/horse/2016104750/', '/horse/2018104980/', '/horse/2018103559/', '/horse/2017105525/', '/horse/2017106203/', '/horse/2016100915/', '/horse/2015105032/', '/horse/2014101976/', '/horse/2017101419/', '/horse/2016104791/', '/horse/2017106711/', '/horse/2014105258/', '/horse/2016105526/', '/horse/2018101071/']


In [127]:
url = "https://db.netkeiba.com/horse/2017102170/"
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")
soup_span = soup.find_all("span")

soup_txt_race = soup.find_all(href = re.compile("/race/20"))
soup_txt_race_str = [str(n) for n in soup_txt_race]
idx=soup_txt_race_str.index('<a href="/race/202106050811/" title="有馬記念(G1)">有馬記念(G1)</a>')
recent_race=[soup_txt_race[idx+1],soup_txt_race[idx+2],soup_txt_race[idx+3]]
print(recent_race)

#過去3回の出走レース名取得
race_list = []
for num in range(0,3):
    try:
        race_list.append(recent_race[num].contents[0])
    except IndexError:
        race_list.append(None)
print(race_list)


[<a href="/race/2021C8100304/" title="凱旋門賞(G1)">凱旋門賞(G1)</a>, <a href="/race/2021C8a00b06/" title="フォワ賞(G2)">フォワ賞(G2)</a>, <a href="/race/202109021211/" title="天皇賞(春)(G1)">天皇賞(春)(G1)</a>]
['凱旋門賞(G1)', 'フォワ賞(G2)', '天皇賞(春)(G1)']


In [62]:
import numpy as np

url = "https://db.netkeiba.com/horse/2017102170/"
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")
soup_span = soup.find_all("span")

soup_txt_race = soup.find_all(href = re.compile("/race/20"))
soup_txt_race_str = [str(n) for n in soup_txt_race]
print(soup_txt_race_str)




['<a href="/race/202109011211/" title="21\'阪神大賞典(G2)">21\'阪神大賞典(G2)</a>', '<a href="/race/202209030411/" title="宝塚記念(G1)">宝塚記念(G1)</a>', '<a href="/race/202209021211/" title="天皇賞(春)(G1)">天皇賞(春)(G1)</a>', '<a href="/race/202209011211/" title="阪神大賞典(G2)">阪神大賞典(G2)</a>', '<a href="/race/202106050811/" title="有馬記念(G1)">有馬記念(G1)</a>', '<a href="/race/2021C8100304/" title="凱旋門賞(G1)">凱旋門賞(G1)</a>', '<a href="/race/2021C8a00b06/" title="フォワ賞(G2)">フォワ賞(G2)</a>', '<a href="/race/202109021211/" title="天皇賞(春)(G1)">天皇賞(春)(G1)</a>', '<a href="/race/202109011211/" title="阪神大賞典(G2)">阪神大賞典(G2)</a>', '<a href="/race/202106010111/" title="日刊スポ賞中山金杯(G3)">日刊スポ賞中山金杯(G3)</a>', '<a href="/race/202008040611/" title="菊花賞(G1)">菊花賞(G1)</a>', '<a href="/race/202007020711/" title="神戸新聞杯(G2)">神戸新聞杯(G2)</a>', '<a href="/race/202005021211/" title="東京優駿(G1)">東京優駿(G1)</a>', '<a href="/race/202008030511/" title="京都新聞杯(G2)">京都新聞杯(G2)</a>', '<a href="/race/202006030811/" title="皐月賞(G1)">皐月賞(G1)</a>', '<a href="/race/202009

In [240]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import time
import pandas
import os

# 列名から部分一致でindexを返す関数の定義
def inclusive_index(lst, purpose):
    for i, e in enumerate(lst):
        if purpose in e: return i

    raise IndexError
    
url = "https://db.netkeiba.com/horse/2017102170/"
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")

# 直近3回の出走レース名の取得
soup_txt_race = soup.find_all(href = re.compile("/race/20"))
soup_txt_race_str = [str(n) for n in soup_txt_race]
idx=soup_txt_race_str.index('<a href="/race/202106050811/" title="有馬記念(G1)">有馬記念(G1)</a>')
recent_race=[soup_txt_race[idx+1],soup_txt_race[idx+2],soup_txt_race[idx+3]]
recent_race_str=[str(n) for n in recent_race]

recent_race_list = []
for num in range(0,3):
    try:
        recent_race_list.append(recent_race[num].contents[0])
    except IndexError:
        recent_race_list.append(None)
print(recent_race_list)

#直近3回の出走レースの詳細情報の取得 
soup_td = soup.find_all('td')
soup_td_str = [str(n) for n in soup_td]

for race in recent_race_str:
    try:
        idx2=inclusive_index(soup_td_str,race)
        recent_race_info = []
        recent_race_info.append(race)
        recent_race_info.append(soup_td[idx2+5].text)
        recent_race_info.append(soup_td[idx2+6].text)
        recent_race_info.append(soup_td[idx2+7].text)
        print(recent_race_info)
    except IndexError:
        print(None)
        


['凱旋門賞(G1)', 'フォワ賞(G2)', '天皇賞(春)(G1)']
['<a href="/race/2021C8100304/" title="凱旋門賞(G1)">凱旋門賞(G1)</a>', '12.8', '6', '14']
['<a href="/race/2021C8a00b06/" title="フォワ賞(G2)">フォワ賞(G2)</a>', '\xa0', '5', '1']
['<a href="/race/202109021211/" title="天皇賞(春)(G1)">天皇賞(春)(G1)</a>', '3.6', '1', '2']


In [207]:
#各年の出走馬のデータ取得

import requests
from bs4 import BeautifulSoup
import csv
import re
import time
import pandas
import os

#関数定義
# 列名から部分一致でindexを返す関数の定義
def inclusive_index(lst, purpose):
    for i, e in enumerate(lst):
        if purpose in e: return i

    raise IndexError


#過去20年のレースIDの取得
raceid_list = []
url = "https://db.netkeiba.com/?pid=race_list&word=%5E%CD%AD%C7%CF%B5%AD%C7%B0"
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")

soup_txt_race = soup.find_all(href = re.compile("/race/20"))
for num in range(20):
    raceid_list.append(soup_txt_race[num].attrs['href'])
    racepara = soup_txt_race[num]
    print(racepara)



<a href="/race/202106050811/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/202006050811/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/201906050811/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/201806050811/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/201706050811/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/201606050910/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/201506050810/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/201406040810/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/201306050810/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/201206050810/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/201106050810/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/201006050810/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/200906050810/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/200806050810/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/200706050809/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/200606050809/" title="有馬記念(G1)">有馬記念(G1)</a>
<a href="/race/200506050809/" title="有馬記